VSTOXX 변동성 지수의 모형으로는 square_root_diffusion 클래스를 사용
& 다음과 같은 점을 고려하여 구현

1. DATA 수집
    - VSTOXX 지수뿐 아니라 이 지수를 기초자산으로 하는 선물과 옵션, 세 가지 유형의 자료가 필요
   
2. 모형 캘리브레이션
    - 시장에서 거래되지 않는 옵션의 가치 평가를 위해선 모형에 사용되는 파라미터들이 현재 거래되는 옵션 가격을 최대한 유사하게 할 수 있는 값을 가지도록 캘리브레이션 해야함

3. 포트폴리오 가치 평가
    - VSTOXX 변동성 지수를 기초자산으로 하고 시장 가격에 캘리브레이션된 자료 및 모형이 있다면 현재 거래되지 않는 옵션을 모형화하고 가치 평가 할 수 있음

# 19.1.1 VSTOXX 지수 데이터 
VSTOXX를 기초자산으로 하는 아메리칸 풋 옵션을 가치 평가하기 위해 필요한 자료 수집

In [1]:
import numpy as np
import pandas as pd

In [2]:
url = 'http://www.stoxx.com/download/historical_values/h_vstoxx.txt'

In [3]:
vstoxx_index = pd.read_csv(url, index_col = 0, header = 2, parse_dates = True, dayfirst = True)

In [4]:
vstoxx_index.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 4357 entries, 1999-01-04 to 2016-02-12
Data columns (total 9 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   V2TX    4357 non-null   float64
 1   V6I1    3906 non-null   float64
 2   V6I2    4357 non-null   float64
 3   V6I3    4296 non-null   float64
 4   V6I4    4357 non-null   float64
 5   V6I5    4357 non-null   float64
 6   V6I6    4340 non-null   float64
 7   V6I7    4357 non-null   float64
 8   V6I8    4343 non-null   float64
dtypes: float64(9)
memory usage: 340.4 KB


2014년 1사분기 자료 사용 -> 그 외 자료는 삭제

In [5]:
vstoxx_index = vstoxx_index[('2013/12/31' < vstoxx_index.index) & (vstoxx_index.index < '2014/4/1')]

자료를 보면 V2TX의 일간 종가뿐 아니라 V6I1에서 V6I8까지의 보조 지수의 자료도 존재.

* 기호의 마지막 숫자는 만기를 표시(1 = 가장 가까운 만기, 8 = 가장 먼 만기)
* 여기서 지수는 V6I1과 V6I2를 내삽한 값(1 : 30일 이내의 만기, 2 : 30일부터 60일 사이의 만기)

In [6]:
np.round(vstoxx_index.tail(), 2)

,V2TX,V6I1,V6I2,V6I3,V6I4,V6I5,V6I6,V6I7,V6I8
Date,,,,,,,,,
2014-03-25,18.26,18.23,18.31,19.04,19.84,20.31,18.11,20.83,21.20
2014-03-26,17.59,17.48,17.70,18.45,19.42,20.00,20.26,20.45,20.86
2014-03-27,17.64,17.50,17.76,18.62,19.49,20.05,20.11,20.49,20.94
2014-03-28,17.03,16.68,17.29,18.33,19.30,19.83,20.14,20.38,20.82
2014-03-31,17.66,17.61,17.69,18.57,19.43,20.04,19.98,20.44,20.90


# 19.1.2 VSTOXX 선물 데이터

In [7]:
vstoxx_futures = pd.read_excel('C:/Users/anrnd/Desktop/파이썬 금융분석/vstoxx_march_2014.xlsx','vstoxx_futures')

C:\Users\anrnd\anaconda3\lib\site-packages\openpyxl\worksheet\_reader.py:312: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


In [8]:
vstoxx_futures.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 504 entries, 0 to 503
Data columns (total 8 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   A_DATE                     504 non-null    datetime64[ns]
 1   A_EXP_YEAR                 504 non-null    int64         
 2   A_EXP_MONTH                504 non-null    int64         
 3   A_CALL_PUT_FLAG            504 non-null    object        
 4   A_EXERCISE_PRICE           504 non-null    int64         
 5   A_SETTLEMENT_PRICE_SCALED  504 non-null    int64         
 6   A_PRODUCT_ID               504 non-null    object        
 7   SETTLE                     504 non-null    float64       
dtypes: datetime64[ns](1), float64(1), int64(4), object(2)
memory usage: 31.6+ KB


In [9]:
del vstoxx_futures['A_SETTLEMENT_PRICE_SCALED']
del vstoxx_futures['A_CALL_PUT_FLAG']
del vstoxx_futures['A_EXERCISE_PRICE']
del vstoxx_futures['A_PRODUCT_ID']

In [10]:
columns = ['DATE', 'EXP_YEAR', 'EXP_MONTH', 'PRICE']
vstoxx_futures.columns = columns

거래소에서 거래되는 옵션 만기일이 매 만기 월의 세번째 금요일 -> 연도와 월 입력하면 세번째 금요일 날짜를 출력해주는 보조 함수 필요

In [11]:
import datetime as dt
import calendar

In [12]:
def third_friday(date):
    day = 21 - (calendar.weekday(date.year, date.month, 1) + 2) % 7
    return dt.datetime(date.year, date.month, day)

In [13]:
# vstoxx 선물이나 옵션은 언제나 현재 달의 세 번째 금요일과 다음 달의 세 번째 금요일을 포함하여 8개의 만기 존재
# 2014년 1월 부터 2014년 11월 까지 11개의 만기 존재

set(vstoxx_futures['EXP_MONTH'])

{1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11}

In [14]:
third_fridays = {}
for month in set(vstoxx_futures['EXP_MONTH']):
        third_fridays[month] = third_friday(dt.datetime(2014, month, 1))

third_fridays

{1: datetime.datetime(2014, 1, 17, 0, 0),
 2: datetime.datetime(2014, 2, 21, 0, 0),
 3: datetime.datetime(2014, 3, 21, 0, 0),
 4: datetime.datetime(2014, 4, 18, 0, 0),
 5: datetime.datetime(2014, 5, 16, 0, 0),
 6: datetime.datetime(2014, 6, 20, 0, 0),
 7: datetime.datetime(2014, 7, 18, 0, 0),
 8: datetime.datetime(2014, 8, 15, 0, 0),
 9: datetime.datetime(2014, 9, 19, 0, 0),
 10: datetime.datetime(2014, 10, 17, 0, 0),
 11: datetime.datetime(2014, 11, 21, 0, 0)}

In [15]:
# 위의 만기일을 DataFrame 객체의 EXP_MONTH 열에 적용
tf = lambda x: third_fridays[x]
vstoxx_futures['MATURITY'] = vstoxx_futures['EXP_MONTH'].apply(tf)

vstoxx_futures.tail()

,DATE,EXP_YEAR,EXP_MONTH,PRICE,MATURITY
499,2014-03-31,2014,7,20.40,2014-07-18
500,2014-03-31,2014,8,20.70,2014-08-15
501,2014-03-31,2014,9,20.95,2014-09-19
502,2014-03-31,2014,10,21.05,2014-10-17
503,2014-03-31,2014,11,21.25,2014-11-21


# 19.1.3 VSTOXX 옵션 데이터

In [16]:
vstoxx_options = pd.read_excel('C:/Users/anrnd/Desktop/파이썬 금융분석/vstoxx_march_2014.xlsx','vstoxx_options')

C:\Users\anrnd\anaconda3\lib\site-packages\openpyxl\worksheet\_reader.py:312: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


In [17]:
vstoxx_options.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46960 entries, 0 to 46959
Data columns (total 24 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   A_DATE                     46960 non-null  datetime64[ns]
 1   A_EXP_YEAR                 46960 non-null  int64         
 2   A_EXP_MONTH                46960 non-null  int64         
 3   A_CALL_PUT_FLAG            46960 non-null  object        
 4   A_EXERCISE_PRICE           46960 non-null  int64         
 5   A_SETTLEMENT_PRICE_SCALED  46960 non-null  int64         
 6   A_PRODUCT_ID               46960 non-null  object        
 7   SETTLE                     46960 non-null  float64       
 8   Unnamed: 8                 0 non-null      float64       
 9   Unnamed: 9                 0 non-null      float64       
 10  Unnamed: 10                0 non-null      float64       
 11  Unnamed: 11                0 non-null      float64       
 12  Unna

In [18]:
del vstoxx_options['A_SETTLEMENT_PRICE_SCALED']
del vstoxx_options['A_PRODUCT_ID']
del vstoxx_options['Unnamed: 8']
del vstoxx_options['Unnamed: 9']
del vstoxx_options['Unnamed: 10']
del vstoxx_options['Unnamed: 11']
del vstoxx_options['Unnamed: 12']
del vstoxx_options['Unnamed: 13']
del vstoxx_options['Unnamed: 14']
del vstoxx_options['Unnamed: 15']
del vstoxx_options['Unnamed: 16']
del vstoxx_options['Unnamed: 17']
del vstoxx_options['Unnamed: 18']
del vstoxx_options['Unnamed: 19']
del vstoxx_options['Unnamed: 20']
del vstoxx_options['Unnamed: 21']
del vstoxx_options['Unnamed: 22']
del vstoxx_options['Unnamed: 23']

In [19]:
columns = ['DATE', 'EXP_YEAR', 'EXP_MONTH', 'TYPE', 'STRIKE', 'PRICE']
vstoxx_options.columns = columns

In [21]:
vstoxx_options['MATURITY'] = vstoxx_options['EXP_MONTH'].apply(tf)
vstoxx_options.head()

,DATE,EXP_YEAR,EXP_MONTH,TYPE,STRIKE,PRICE,MATURITY
0,2014-01-02,2014,1,C,1000,7.95,2014-01-17
1,2014-01-02,2014,1,C,1500,3.05,2014-01-17
2,2014-01-02,2014,1,C,1600,2.20,2014-01-17
3,2014-01-02,2014,1,C,1700,1.60,2014-01-17
4,2014-01-02,2014,1,C,1800,1.15,2014-01-17


하나의 옵션 계약은 지수 값의 100배 -> 행사가 조정 필요

In [23]:
vstoxx_options['STRIKE'] = vstoxx_options['STRIKE'] / 100.

In [24]:
save = False
if save is True:
    import warnings
    warnings.simplefilter('ignore')
    h5 = pd.HDFStore('C:/Users/anrnd/Desktop/파이썬 금융분석/vstoxx_march_2014.h5', vomplevel = 9, complib = 'blosc')
    h5['vstoxx_index'] = vstoxx_index
    h5['vstoxx_futures'] = vstoxx_futures
    h5['vstoxx_options'] = vstoxx_options
    h5.close()